# Simplicial Groups

## Representation of a  simplicial group

A simplicial group is  an instance of the class `SIMPLICIAL-GROUP`, subclass of the classes `KAN` and `HOPF-ALGEBRA`.
```commonlisp
(DEFCLASS SIMPLICIAL-GROUP (kan hopf-algebra)
   ((grml :type simplicial-mrph :initarg :grml :reader grml1)
    (grin :type simplicial-mrph :initarg :grin :reader grin1)))
```
We recall that this new class (multi-) inherits from the following classes: `KAN`, `SIMPLICIAL-SET`, `COALGEBRA`,  `ALGEBRA` and `CHAIN-COMPLEX`. It has two slots of its own:

`grml`, an object of type `SIMPLICIAL-MRPH`, defining in the underlying simplicial set $S$, the group operation as a simplicial morphism from $S \times S$ onto $S$, compatible with the face and degeneracy operators in $S$, i.e. the $\partial$'s and $\eta$'s operators are also group morphisms. In dimension $n$, the neutral element is assumed to be the $n$-th degeneracy of the base point of the underlying simplicial set.

`grin`, an object of type `SIMPLICIAL-MRPH`, defining the inverse of an element of the Kan simplicial set w.r.t. the preceding group law.

A printing method has been associated to the class `SIMPLICIAL GROUP` and the external representation of an instance is a string like `[K`*n* `Simplicial-Group]`, where *n* is the number plate of the Kenzo object.

## Representation of an Abelian  simplicial group

The class of Abelian simplicial group, `AB-SIMPLICIAL-GROUP`, inherits  all the properties of the class `SIMPLICIAL-GROUP` and has not slots of its own, as shown by:
```commonlisp
(DEFCLASS AB-SIMPLICIAL-GROUP (simplicial-group) ())
```
The difference is purely mathematical: it is up to the user to provide as slot *grml*, a simplicial morphism which is **commutative**. As expected, the external representation of  an instance of this class, is a string like
`[K`*n* `Abelian-Simplicial-Group]`, where *n* is the number plate of the Kenzo object.

## The functions `build-smgr` and `build-ab-smrg`

To facilitate the construction of instances of the classes `SIMPLICIAL-GROUP` and `AB-SIMPLICIAL-GROUP` and to free  the user to call the standard constructor `make-instance`, the software provides the functions *build-smrg* and *build-ab-smrg*.

`build-smgr`
> `:cmpr` *cmpr* `:basis` *basis* `:bspn` *bspn* `:face` *face* `:face*` *face*$*$  
`:intr-bndr` *intr-bndr* `:bndr-strt` *bndr-strt* `:intr-dgnl` *intr-dgnl*  
`:dgnl-strt` *dgnl-strt* `:sintr-grml` *sintr-grml* `:sintr-grin` *sintr-grin*  
`:orgn` *orgn*

defined with keyword parameters and returning an instance of the class `SIMPLICIAL-GROUP`. The keyword arguments *cmpr*, *basis*, *bspn*, *face*, *face*$*$, *intr-bndr*, *bndr-strt*, *intr-dgnl* and *dgnl-strt* are the arguments provided to build the underlying simplicial set using the function `build-smst`. Let us call it $S$. As usual, an adequate comment list must be provided for the parameter `:orgn`. The only two new arguments are:

- *sintr-grml*, a lisp function defining the group operation $S \times S \longrightarrow S$. The function `build-smgr` uses this lisp function to build a simplicial morphism of degree 0 between the cartesian product of the newly created simplicial set $S$ and itself. This new simplicial morphism is the value of the slot `grml` of the simplicial group instance.
- *sintr-grin*, a lisp function defining the inverse w.r.t the preceding group law of an element of $S$. The function `build-smgr` uses this lisp function to build a simplicial morphism of degree 0 between $S$ and itself. This new simplicial morphism is the value of the slot `grin` of the simplicial group instance.

After a call to `build-smgr`, the simplicial group instance is pushed onto the list of previously constructed  ones (`*smgr-list*`). The simplicial group with Kenzo identification $n$, may be retrieved in the `*smgr-list*`, by a call to the function `sgmr` as `(sgmr` *n*`)`. The list `*smgr-list*` may be cleared by the function `cat-init`.

Up to now the slot `kfll` has not been filled. As long as the user does not make use of the filling function, directly or indirectly, this slot remains unbound. But as soon as it is needed, the *slot-unbound* mechanism of CLOS enters in action and the filling function is defined by the system with the help of the functions `face`, `sintr-grml` and `sintr-grin`. The automatic creation of the filling function is possible because mathematically, a simplicial group is always a Kan simplicial set. This is realized by the internal function `smgr-kfll-intr`.

For the user, the two simplicial morphisms, values of the specific slots  of a `SIMPLICIAL-GROUP` are applied via the two following macros:

`grml` *smgr dmns crpr* *[Macro]*
> Apply the simplicial morphism, value of the slot `grml` of the simplicial group instance *smgr* to the cartesian product *crpr* in dimension *dmns*. Mathematically, this realizes the group operation. In principle, the simplicial morphism must accept the *crpr* argument on both forms: either a geometric cartesian product or an abstract simplex, the geometric part of which is a geometric cartesian product (see the examples). With only one argument (*smgr*) the macro selects the simplicial morphism instance.

`grin` *smgr dmns absm-or-gsm* *[Macro]*
> Apply the simplicial morphism, value of the slot `grin` of the simplicial group instance *smgr* to the element *absm-or-gsm* in dimension *dmns*. Mathematically this gives the inverse in the group of the element. As suggested by the name of the parameter, this must work with geometric simplices as well as abstract simplices. With only one argument (*smgr*) the macro selects the simplicial morphism.

`build-ab-smgr`
> `:cmpr` *cmpr* `:basis` *basis* `:bspn` *bspn* `:face` *face* `:face*` *face*$*$  
`:intr-bndr` *intr-bndr* `:bndr-strt` *bndr-strt* `:intr-dgnl` *intr-dgnl*  
`:dgnl-strt` *dgnl-strt* `:sintr-grml` *sintr-grml* `:sintr-grin` *sintr-grin*  
`:orgn` *orgn*

The description of the parameters is exactly the same as in the previous function, but in this case, the group operation given by the user, (argument *sintr-grml*), must be commutative. The lisp definition of `build-ab-smgr` is simply:
```commonlisp
(DEFMACRO BUILD-AB-SMGR (&rest rest)
  `(change-class (build-smgr ,@rest) 'ab-simplicial-group))
```

## Two important Abelian simplicial groups: $K(\mathbb{Z}, 1)$ and $K(\mathbb{Z}_2, 1)$}

Let us recall the notion of classifying space in the framework of simplicial sets and in the particular case of discrete groups. Given a discrete group $\cal G$, we know that we may build a simplicial set $K({\cal G}, 1)$; in dimension $n$, we have $(K({\cal G}, 1))_n = {\cal G}^n$, the simplices or *bar objects* being  conventionally represented  as sequences of elements of $\cal G$:
$$ [g_1 \mid g_2 \mid \ldots \mid g_n]. $$
The base point is the void bar object $[\ ]$. Let us note the group law multiplicatively.

The face operators are defined as follows:
$$
\begin{aligned}
\partial_0 [g_1 \mid g_2 \mid \ldots \mid g_n] & = & [g_2 \mid g_3 \mid \ldots \mid g_n], \\
\partial_1 [g_1 \mid g_2 \mid \ldots \mid g_n] & = & [{\bf g_1g_2} \mid g_3 \mid \ldots \mid g_n], \\
\partial_2 [g_1 \mid g_2 \mid \ldots \mid g_n] & = & [g_1 \mid {\bf g_2g_3} \mid \ldots \mid g_n], \\
                        ......                & = &  ......  \\
\partial_{n-1} [g_1 \mid g_2 \mid \ldots \mid g_n] & = & [g_1 \mid g_2 \mid \ldots \mid g_{n-2} \mid {\bf g_{n-1}g_n}], \\
\partial_n [g_1 \mid g_2 \mid \ldots \mid g_n] & = & [g_1 \mid g_2 \mid \ldots \mid g_{n-1}].
\end{aligned}
$$
The degeneracy operators are defined as follows:
$$ \eta_i [g_1 \mid \ldots \mid g_{i-1} \mid g_i \mid g_{i+1} \mid \ldots \mid g_n] =
          [g_1 \mid \ldots \mid g_{i-1}  \mid g_i \mid {\bf e} \mid g_{i+1} \mid \ldots \mid g_n],$$
where $e$ is the neutral element of $\cal G$.
\par
If, in addition $\cal G$ is abelian, then  $K({\cal G}, 1)$ is itself a *simplicial group*, with the
following group law, in dimension $n$:
$$[a_1  \mid \cdots \mid a_n] . [b_1 \mid \cdots \mid b_n] = [a_1+b_1 \mid \cdots \mid a_n+b_n], $$
$$[a_1  \mid \cdots \mid a_n]^{-1}= [-a_1  \mid \cdots \mid -a_n],\quad a,b \in {\cal G}$$
In dimension $n$, the neutral element is the bar object constitued by a sequence of $n$ times the neutral $e$ of ${\cal G}$; this is also the $n$-th degeneracy of the base point $[\ ]$ of $K({\cal G}, 1)$.

The two important simplicial groups $K(\mathbb{Z}, 1)$ and $K(\mathbb{Z}_2, 1)$ may be constructed in `Kenzo`.

`k-z-1` *[Function]*
> Build the simplicial group $K(\mathbb{Z},1)$. In this simplicial group, a  simplex  in dimension $n$ is mathematically represented by a sequence of integers, known as a *bar* object:
$$ [a_1 \mid a_2 \mid \ldots \mid a_n]. $$
The group operation inherits the additive law of $\mathbb{Z}$:
$$[a_1  \mid \cdots \mid a_n] + [b_1 \mid \cdots \mid b_n] = [a_1+b_1 \mid \cdots \mid a_n+b_n]$$
and the inverse of $[a_1  \mid \ldots \mid a_n]$ is $[-a_1 \mid \ldots \mid -a_n]. $
In `Kenzo`, a non-degenerate simplex of $K(\mathbb{Z},1)$ in dimension $n$ will be simply a list of $n$ non-null integers. The underlying simplicial set is locally effective and its base point is `NIL`, i.e the void bar object
$[\ ]$. To be coeherent with the general policy of `Kenzo`, a degenerate simplex must be represented by
an abstract simplex of the form `(:absm` *dgop* `.` *gmsm*`)` where *gmsm* is a non-degenerate simplex, here a list of non-null integers. But, for the computations in the simplicial group, the general form of a bar object, including 0, is more natural and more convenient. Consequently, two conversion functions are provided (see below).

`k-z2-1` *[Function]*
> Build the simplicial group $K(\mathbb{Z}_2,1)$. Mathematically, in dimension $n$, the only non-degenerate simplex is
a sequence of $n$ 1's. But one may show that as simplicial set, $K(\mathbb{Z}_2,1)$ is  isomorphic to $P^\infty \mathbb{R}$ which may be built by the statement `(R-proj-space)`. This is the representation adopted in `Kenzo`. In dimension $n$, this simplicial set has only one non-degenerate simplex, namely the integer $n$. The formulas for the faces of this non-degenerate simplex $n$ collapse to:
$$
\begin{aligned}
\partial_0 n & = & n-1, \\
\partial_i n & = & \eta_{i-1} (n-2), \quad i \not= 0, i \not=n,  \\
\partial_n n & = & n-1.
\end{aligned}
$$

As a group, in dimension $n$, the non-degenerate element $n$ is its own inverse and the neutral is the $n$-th degeneracy of 0. As in $K(\mathbb{Z}, 1)$ a degenerate simplex must be represented by a valid abstract simplex, but as the computations are more convenient if the general bar objects are represented by a sequence of mixed 0's and 1's, two conversion functions are
provided.

`z-bar-absm` *bar* *[Function]*
> Transform a general mathematical bar object *bar* of $K(\mathbb{Z}, 1)$, represented by a list of integers (including 0's), into a valid abstract simplex (an object of type `ABSM`), `(:absm` *dgop* `.` *gmsm*`)`, where *gmsm*  is a sequence of non-null integers representing a non-degenerate bar object of $K(\mathbb{Z},1)$ and *dgop* a coded sequence of degeneracy operators $\eta_i$'s.

`z-absm-bar` *absm* *[Function]*
> Transform the abstract simplex *absm*,  `(:absm` *dgop* `.` *gmsm*`)`, where *gmsm* is a sequence of non-null integers representing a nondegenerate bar object of $K(\mathbb{Z},1)$ and *dgop* a coded sequence of degeneracy operators, into a list of integers representing a bar object, degenerate or not, of $K(\mathbb{Z},1)$, for more convenience in the internal computations or for more clarity in external printing.

`z2-bar-absm` *bar* *[Function]*
> Function analogous to `z-bar-absm` but for  $K(\mathbb{Z}_2, 1)$.

`z2-absm-bar` *absm* *[Function]*
> Function analogous to `z-absm-bar` but for  $K(\mathbb{Z}_2, 1)$.

### Examples

In [1]:
(def KZ1 (k-z-1))

[K1 Abelian-Simplicial-Group]

In [2]:
(def simplex '(1 10 100))

(1 10 100)

In [3]:
(face KZ1 0 3 simplex)

<AbSm - (10 100)>

Let us build the list of the four faces and by suppressing successively, each face one after the other in this list, let us verify that we have always a Kan hat (we recall that the lisp function `remove`, works on a copy of the list, i.e. the argument `hat` is not modified). In the forth  statement, we verify the property of minimality of this special simplicial group.

In [4]:
(def hat (mapcar #'(lambda (i) (face KZ1 i 3 simplex)) (<a-b> 0 3)))

(<AbSm - (10 100)> <AbSm - (11 100)> <AbSm - (1 110)> <AbSm - (1 10)>)

In [6]:
(dotimes (i 4)
    (check-kan KZ1 i 3 (remove (nth i hat) hat)))


---done---
---done---
---done---
---done---

NIL

In [7]:
(def hat1 (remove (nth 1 hat) hat))

(<AbSm - (10 100)> <AbSm - (1 110)> <AbSm - (1 10)>)

In [8]:
(kfll KZ1 1 3 hat1)

<AbSm - (1 10 100)>

We test now the two macros `grml` and `grin` related to the group operation.

In [9]:
(grml KZ1 3 (crpr 0 simplex 0 simplex))

<AbSm - (2 20 200)>

In [10]:
(def invsimplex (grin KZ1 3 simplex))

<AbSm - (-1 -10 -100)>

In [11]:
(grin KZ1 3 *) 

<AbSm - (1 10 100)>

In [13]:
(grml KZ1 3 (crpr 0 simplex 0 (gmsm invsimplex)))

<AbSm 2-1-0 NIL>

In [14]:
(2absm-acrpr (absm 0 simplex ) (grin KZ1 3 simplex))

<AbSm - <CrPr - (1 10 100) - (-1 -10 -100)>>

In [15]:
(grml KZ1 3 *) 

<AbSm 2-1-0 NIL>

In [16]:
(def KZ2 (k-z2-1))

[K13 Abelian-Simplicial-Group]

In [17]:
(grml KZ2 3 (crpr 0 3 0 3))

<AbSm 2-1-0 0>

In [18]:
(grin KZ2 4 4)

<AbSm - 4>

In [19]:
(grml KZ2 4 (crpr 0 * 0 *))

<AbSm 3-2-1-0 0>

Let us show some examples of conversions, between general bar objects and valid abstract simplices:

In [20]:
(z-absm-bar (absm 0 '()))

NIL

In [21]:
(z-absm-bar (absm 1 '()))

(0)

In [22]:
(z-absm-bar (absm 0 '(2)))

(2)

In [23]:
(z2-absm-bar (absm 7 7))

(0 0 0 1 1 1 1 1 1 1)

In [24]:
(z2-absm-bar (absm 0 7))

(1 1 1 1 1 1 1)

Let us suppose that the value of the symbol `labsms` is a list of abstract simplices built from the simplex `(3 6)` in dimension 1 of $K(\mathbb{Z}, 1)$. The application of the function `z-absm-bar` on this list gives the representation of the simplices under the classical mathematical representation. Applying then `z-bar-absm` returns the original list. A similar example is given with the symbol `labsm2` in dimension 0.

In [ ]:
#|
labsms ==>

(<AbSm - (3 6)> <AbSm 0 (3 6)> <AbSm 1 (3 6)>
 <AbSm 1-0 (3 6)> <AbSm 2 (3 6)> <AbSm 2-0 (3 6)>
 <AbSm 2-1 (3 6)> <AbSm 2-1-0 (3 6)>)

(mapcar #'z-absm-bar labsms)  ==>

((3 6) (0 3 6) (3 0 6) (0 0 3 6) (3 6 0) (0 3 0 6) (3 0 0 6) (0 0 0 3 6))

(mapcar #'z-bar-absm *)  ==>

(<AbSm - (3 6)> <AbSm 0 (3 6)> <AbSm 1 (3 6)>
 <AbSm 1-0 (3 6)> <AbSm 2 (3 6)> <AbSm 2-0 (3 6)>
 <AbSm 2-1 (3 6)> <AbSm 2-1-0 (3 6)>)

labsms2   ==>

(<AbSm - 0> <AbSm - 1> <AbSm - 2> <AbSm - 3>
 <AbSm 0 0> <AbSm 0 1> <AbSm 0 2> <AbSm 0 3>
 <AbSm 1-0 0> <AbSm 1-0 1> <AbSm 1-0 2> <AbSm 1-0 3>)

(mapcar #'z2-absm-bar labsms2)  ==>

(NIL (1) (1 1) (1 1 1) (0) (0 1) (0 1 1) (0 1 1 1) (0 0) (0 0 1) (0 0 1 1)
 (0 0 1 1 1))

(mapcar #'z2-bar-absm *)  ==>

(<AbSm - 0> <AbSm - 1> <AbSm - 2> <AbSm - 3>
 <AbSm 0 0> <AbSm 0 1> <AbSm 0 2> <AbSm 0 3>
 <AbSm 1-0 0> <AbSm 1-0 1> <AbSm 1-0 2> <AbSm 1-0 3>)
|#

## Simplicial Groups as Algebras

If we consider a simplicial group ${\cal G}$, with group operation $\tau$, there is a canonical product for the algebra
$$\varpi : C_*({\cal G}) \otimes C_*({\cal G}) \longrightarrow C_*({\cal G}), $$
defined as the composition $\tau \circ {\cal EML}$, where ${\cal EML}$ is the Eilenberg-Mac Lane homomorphism or the *shuffle homomorphism* (see the Eilenberg-Zilber chapter):
$$C_*({\cal G}) \otimes C_*({\cal G}) \buildrel {{\cal EML}} \over \longrightarrow
   C_*({\cal G} \times {\cal G}) \buildrel {\tau} \over \longrightarrow C_*({\cal G}). $$
For instance let us look at the simplicial group $K(\mathbb{Z}, 1)$:

In [25]:
(def kz1 (k-z-1))

[K1 Abelian-Simplicial-Group]

In [26]:
(inspect kz1)


The object is a STANDARD-OBJECT of type AB-SIMPLICIAL-GROUP.
0. CMPR: #<FUNCTION K-Z-1-CMPR>
1. BASIS: :LOCALLY-EFFECTIVE
2. BSGN: NIL
3. DFFR: [K2 Morphism (degree -1): K1 -> K1]
4. GRMD: [K1 Abelian-Simplicial-Group]
5. EFHM: "unbound"
6. IDNM: 1
7. ORGN: (K-Z-1)
8. APRD: "unbound"
9. CPRD: [K5 Morphism (degree 0): K1 -> K3]
10. FACE: #<FUNCTION K-Z-1-FACE>
11. KFLL: #<CLOSURE (FLET CAT::RSLT :IN CAT::SMGR-KFLL-INTR) {100576B5DB}>
12. GRML: [K11 Simplicial-Morphism K6 -> K1]
13. GRIN: [K12 Simplicial-Morphism K1 -> K1]

> 

NIL

We see that  the slot `aprd` has not been filled. As long as the user does not  need, directly or indirectly, the product of the algebra, this slot remains unbound. But as soon as it is needed, the *slot-unbound* mecanism of CLOS enters in action, the canonical algebra product morphism is defined by the system, and the slot
`aprd` is set.

For instance, let us take the tensor product of two elements of  $K(\mathbb{Z}, 1)$ and apply the product in the algebra:

In [27]:
(def tnsrp (tnpr 2 '(1 2) 3 '(7 8 9))) 

<TnPr (1 2) (7 8 9)>

In [28]:
(aprd kz1 5 tnsrp)


----------------------------------------------------------------------{CMBN 5}
<1 * (1 2 7 8 9)>
<-1 * (1 7 2 8 9)>
<1 * (1 7 8 2 9)>
<-1 * (1 7 8 9 2)>
<1 * (7 1 2 8 9)>
<-1 * (7 1 8 2 9)>
<1 * (7 1 8 9 2)>
<1 * (7 8 1 2 9)>
<-1 * (7 8 1 9 2)>
<1 * (7 8 9 1 2)>
------------------------------------------------------------------------------


The system `Kenzo` has automatically created the needed algebra product morphism  and this is shown in the slot instance `aprd`:

In [29]:
(inspect kz1)


The object is a STANDARD-OBJECT of type AB-SIMPLICIAL-GROUP.
0. CMPR: #<FUNCTION K-Z-1-CMPR>
1. BASIS: :LOCALLY-EFFECTIVE
2. BSGN: NIL
3. DFFR: [K2 Morphism (degree -1): K1 -> K1]
4. GRMD: [K1 Abelian-Simplicial-Group]
5. EFHM: "unbound"
6. IDNM: 1
7. ORGN: (K-Z-1)
8. APRD: [K26 Morphism (degree 0): K3 -> K1]
9. CPRD: [K5 Morphism (degree 0): K1 -> K3]
10. FACE: #<FUNCTION K-Z-1-FACE>
11. KFLL: #<CLOSURE (FLET CAT::RSLT :IN CAT::SMGR-KFLL-INTR) {100576B5DB}>
12. GRML: [K11 Simplicial-Morphism K6 -> K1]
13. GRIN: [K12 Simplicial-Morphism K1 -> K1]

> 

NIL

An interesting example is to show the associativity of the canonical product of an algebra, by defining in `Kenzo` the morphisms of the following diagram and verifying that this diagram is commutative. In the diagram, $\nabla$ is the algebra product of ${\cal A}$, $1$ is the identity morphism on ${\cal A}$ and
*assoc*, the morphism
$$assoc: ({\cal A}\otimes {\cal A}) \otimes {\cal A} \longrightarrow {\cal A} \otimes ({\cal A}\otimes {\cal A}),$$
with:
$$assoc((a\otimes b)\otimes c) = a \otimes (b \otimes c).$$

$$
\begin{aligned}
& ({\cal A}\otimes {\cal A}) \otimes {\cal A} & \buildrel {assoc} \over \longrightarrow &
  {\cal A} \otimes ({\cal A}\otimes {\cal A}) \cr
&  {\scriptstyle \nabla \otimes 1}\downarrow &          &  \downarrow {\scriptstyle 1 \otimes \nabla}  \cr
& {\cal A}\otimes {\cal A}    &          &  {\cal A}\otimes {\cal A} \cr
& {\scriptstyle \nabla}\searrow             &          &  \swarrow {\scriptstyle \nabla}  \cr
&                             & {\cal A} &  \cr
\end{aligned}
$$